In [ ]:
%load_ext autoreload
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from dbmaster import MasterParams, MasterPersist
from dbbase import MusicDBIDModVal, MusicDBDir, MusicDBData
from dbnote import DownloadRecord, MergeSearchArtist, ConcatRawData
from utils import FileIO, DirInfo, FileInfo, getFlatList, Timestat, TermTime, TermTimeTS, getTT
from pandas import Series, DataFrame, concat, Timestamp
from pandb import PanDBIO
from musicdb.bandcamp import MusicDBParams, RawWebData, MusicDBIO
from os import getpid
from functools import partial

mv = MusicDBIDModVal()
io = FileIO()
mpar = MusicDBParams()
dbio = MusicDBIO()
webio = RawWebData()
db = mpar.db

In [ ]:
searchArtistRecord = DownloadRecord(db=db, name="SearchArtist", rTypes=["Index", "Data"])
downloadArtistRecord = DownloadRecord(db=db, name="DownloadArtist", rTypes=["Index"])
downloadArtistRecommendedRecord = DownloadRecord(db=db, name="DownloadArtistRecommended", rTypes=["Index"])
downloadArtistMusicRecord = DownloadRecord(db=db, name="DownloadArtistMusic", rTypes=["Index"])
knownArtists = dbio.rdio.getSummaryNameData
searchArtists = partial(dbio.rdio.getData, "SearchArtist")

In [ ]:
##########################################################################################
# Show Summary
##########################################################################################
print(f"{db} Search Results (PID={getpid()})".format(db))
searchArtistRecord.info()
downloadArtistRecord.info()
downloadArtistRecommendedRecord.info()
downloadArtistMusicRecord.info()
print(f"  {'KnownArtist Data': <20}: {knownArtists().shape[0]}")

# Search For New Artists

In [ ]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=False)
webio = RawWebData(debug=False)
knownNames = PanDBIO().getUniqueArtistNames()
searchedNames = Series(searchArtistRecord.getIndex())
artistNamesToGet = knownNames[~knownNames.isin(searchedNames.index)]

print(f"# {db} Search Results")
print(f"#   Available Names:     {knownNames.shape[0]}")
print(f"#   Known Artist Names:  {searchedNames.shape[0]}")
print(f"#   Artist Names To Get: {artistNamesToGet.shape[0]}")

del searchedNames
del knownNames

In [ ]:
ts = Timestat(f"Getting {db} ArtistIDs")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def isError(artistName, nErrors, sleeptime, error=None):
    searchArtistRecord.setError(index=artistName)    
    print(f"Search Error ==> {artistName}: {error}")
    nErrors.append(artistName)
    webio.sleep(sleeptime)
    

n = 0
maxN = 2500000
nErrors = []
searchArtistRecord.load()
for i, (idx, artistName) in enumerate(artistNamesToGet.items()):
    if searchArtistRecord.isKnown(artistName):
        continue

    if len(nErrors) >= 5:
        print("Stopping due to 5 consecutive errors")
        break

    try:
        response = webio.getArtistSearchData(artistName=artistName)
    except Exception as error:
        isError(artistName, nErrors, 10, error)
        continue

    if not isinstance(response, list):
        isError(artistName, nErrors, 3.5, "NotList")
        continue

    nErrors = []
    searchArtistRecord.setData(index=artistName, data=response)
    webio.sleep(4.0)
    n += 1
        
    if n % 25 == 0:
        ts.update(n=n)
        searchArtistRecord.save()
        webio.wait(5.0)
        if tt.isFinished():
            break
    
    if n >= maxN:
        print("Breaking after {0} downloads...".format(maxN))
        break

ts.stop()
searchArtistRecord.save()

## Append New Data To Global Search Artist Data

In [ ]:
from utils import flattenLists
    
######################################################################################################
# Explode Recent Data
######################################################################################################
def getNewData(searchArtistRecord):    
    newData = DataFrame(flattenLists(searchArtistRecord.recordData['Data'].values()))
    newData.index = newData["URL"].map(dbio.getdbid)
    newData.index.name = ""
    return newData

newData = getNewData(searchArtistRecord)
msr = MergeSearchArtist(db)
msr.merge(searchArtistRecord, newData, test=False)

# Download Artist Data

In [ ]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=False)
webio = RawWebData(debug=False)
artistNames = searchArtists()
artistNames['IndexModVal'] = artistNames.index.map(mv.getModVal)
artistNames = artistNames[artistNames['IndexModVal'] == 0]
downloadArtistRecord.load(verbose=False)
availableNames = artistNames #[~artistNames.index.map(downloadArtistRecord.isKnown)]
artistNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby(["IndexModVal"])})

print(f"# {db} Download Results")
print(f"#   Available Names:     {artistNames.shape[0]}")
print(f"#   Known Artist Names:  {len(downloadArtistRecord.getIndex())}")
print(f"#   Artist Names To Get: {availableNames.shape[0]}")

del availableNames
del artistNames

In [ ]:
ts = Timestat(f"Getting {db} Artists")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def isError(artistName, nErrors, sleeptime, error=None):
    downloadArtistRecord.setError(index=artistName)    
    print(f"Search Error ==> {artistName}: {error}")
    nErrors.append(artistName)
    webio.sleep(sleeptime)
    

stop = False
n = 0
maxN = 2500000
nErrors = []
downloadArtistRecord.load(verbose=False)
for groupModVal, df in artistNamesToGet.items():
    modVal = groupModVal[0]
    if stop is True:
        break
    N = df.shape[0]
    for i, (artistID, row) in enumerate(df.iterrows()):
        #if downloadArtistRecord.isKnown(artistID):
        #    continue
        if dbio.rdio.getFilename("RawArtist", modVal, artistID).exists():
            print(f"{i} / {artistID} exists")
            continue

        if len(nErrors) >= 5:
            print("Stopping due to 5 consecutive errors")
            stop = True
            break

        artistName = row["Name"]
        print(f"{modVal: <8} | {i: <8} | {N: <8} | {n: <8} | ", end="")

        try:
            response = webio.getArtistData(artistName=artistName, artistID=artistID)
        except Exception as error:
            isError(artistID, nErrors, 10, error)
            continue
    
        if not isinstance(response, bytes):
            isError(artistID, nErrors, 5.0, "NotBytes")
            continue
    
        nErrors = []
        downloadArtistRecord.setIndex(index=artistID)
        dbio.rdio.saveData("RawArtist", modVal, artistID, data=response)
        webio.sleep(4.5)
        n += 1
            
        if n % 25 == 0:
            ts.update(n=n)
            downloadArtistRecord.save()
            webio.wait(10.0)
            if tt.isFinished():
                stop = True
                break
        
        if n >= maxN:
            print("Breaking after {0} downloads...".format(maxN))
            stop = True
            break

ts.stop()
downloadArtistRecord.save()

In [ ]:
downloadArtistRecord.save()

In [ ]:
from dbnote import ConcatRawData
crd = ConcatRawData("Bandcamp", "Artist")
#crd.concat()
#crd.merge(test=False)
crd.remove(force=True)

# Download Artist Music

In [ ]:
def getMusicData():
    dbio = MusicDBIO(verbose=False,local=False,mkDirs=False)
    nameData = dbio.rdio.getSummaryNameData()
    urlData = dbio.rdio.getSummaryRefData()
    musicData = dbio.rdio.getSummaryLinkData()['Tabs'].map(lambda x: x.get('/music'))
    musicURL = urlData.map(lambda x: f"{x}/music")
    musicURL.name = "URL"
    tmp = DataFrame(nameData).join(musicURL)
    refIdx = musicData[musicData.notna()].index
    retval = tmp[tmp.index.isin(refIdx)]
    return retval

In [ ]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=False)
webio = RawWebData(debug=False)
artistNames = getMusicData()
artistNames['IndexModVal'] = artistNames.index.map(mv.getModVal)
artistNames = artistNames[artistNames['IndexModVal'] == 0]
downloadArtistMusicRecord.load(verbose=False)
#availableNames = artistNames[~artistNames.index.map(downloadArtistMusicRecord.isKnown)]
availableNames = artistNames #[~artistNames.index.map(downloadArtistMusicRecord.isKnown)]
artistNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby(["IndexModVal"])})

print(f"# {db} Search Results (PID={getpid()})")
print(f"#   Available Names:     {artistNames.shape[0]}")
print(f"#   Known Artist Names:  {downloadArtistMusicRecord.numKnown()}")
print(f"#   Artist Names To Get: {availableNames.shape[0]}")

del availableNames
del artistNames

In [ ]:
ts = Timestat(f"Getting {db} Artists")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def isError(artistName, artistID, nErrors, sleeptime, error=None):
    downloadArtistMusicRecord.setError(index=artistID)    
    print(f"Search Error ==> {artistName}: {error}")
    nErrors.append(artistName)
    webio.sleep(sleeptime)
    

stop = False
n = 0
maxN = 250000000
nErrors = []
downloadArtistMusicRecord.load(verbose=False)
for groupModVal, df in artistNamesToGet.items():
    modVal = groupModVal[0]
    if stop is True:
        break
    N = df.shape[0]
    for i, (artistID, row) in enumerate(df.iterrows()):
        if i < 831:
            continue
        if dbio.rdio.getFilename("RawArtistMusic", modVal, artistID).exists():
            print(f"{i} / {artistID} exists")
            continue
        if downloadArtistMusicRecord.isError(artistID):
            continue
    
        if len(nErrors) >= 25:
            print("Stopping due to 5 consecutive errors")
            stop = True
            break

        artistName = row["Name"]
        artistURL = row["URL"]
        print(f"{modVal: <8} | {i: <8} | {N: <8} | {n: <8} | ", end="")

        try:
            response = webio.getArtistMusicData(artistID=artistID, artistName=artistName, artistURL=artistURL)
        except Exception as error:
            isError(artistName, artistID, nErrors, 10, error)
            continue
    
        if not isinstance(response, bytes):
            isError(artistName, artistID, nErrors, 5.0, "NotBytes")
            continue
    
        nErrors = []
        downloadArtistMusicRecord.setIndex(index=artistID)
        dbio.rdio.saveData("RawArtistMusic", modVal, artistID, data=response)
        webio.sleep(4.5)
        n += 1
            
        if n % 25 == 0:
            ts.update(n=n)
            downloadArtistMusicRecord.save()
            webio.wait(10.0)
            if tt.isFinished():
                stop = True
                break
        
        if n >= maxN:
            print("Breaking after {0} downloads...".format(maxN))
            stop = True
            break

ts.stop()
downloadArtistMusicRecord.save()

In [ ]:
for modVal in range(3,100):
    crd = ConcatRawData(db=dbio.db, dType="ArtistMusic")
    crd.concat(modVal=modVal)
    crd.merge(modVal=modVal, test=False)
    crd.remove(modVal=modVal, force=True)

# Download Recommended Data

In [ ]:
def getMusicData():
    dbio = MusicDBIO(verbose=False,local=False,mkDirs=False)
    nameData = dbio.rdio.getSummaryNameData()
    urlData = dbio.rdio.getSummaryRefData()
    musicData = dbio.rdio.getSummaryLinkData()['Tabs'].map(lambda x: x.get('/music'))
    musicURL = urlData.map(lambda x: f"{x}/music")
    musicURL.name = "URL"
    tmp = DataFrame(nameData).join(musicURL)
    refIdx = musicData[musicData.notna()].index
    retval = tmp[tmp.index.isin(refIdx)]
    return retval

In [ ]:
dbio = MusicDBIO(verbose=False,local=False,mkDirs=False)
nameData = dbio.rdio.getSummaryNameData()
urlData = dbio.rdio.getSummaryRefData()
musicData = dbio.rdio.getSummaryLinkData()['Tabs'] #.map(lambda x: x.get('/music'))


In [ ]:
dbio = MusicDBIO(verbose=False,local=False,mkDirs=False)

In [ ]:
files = dbio.rdio.getDir("RawArtistMusicModVal", 0).getFiles()

In [ ]:
from utils import getFile
for ifile in files:
    for artistID, aidData in getFile(ifile).items():
        break
    break

In [ ]:
from utils import getHTML
bsdata = getHTML(aidData)

In [ ]:
retval = dbio.rawio.getArtistData(fid='arcadianchildband', data=bsdata, ifile=None)

In [ ]:
retval['Artist'].show()

In [ ]:
for script in bsdata.findAll("script"):
    print("\n\n")
    print(script)
    for attr, attrVal in script.attrs.items():
        print(f"\t{attr: <25}{attrVal}")


In [ ]:
bsdata

In [ ]:
[li for li in bsdata.findAll("li") if li.get('data-item-id') is not None]

In [ ]:
bsdata.findAll("ol", {"class": "music-grid"})

# Download Album Data

## Create Media Data

In [ ]:
mediaData = {}
for modVal in range(100):
    modValData = mio.data.getModValData(modVal)
    modValMediaData = {}
    for artistID,artistIDData in modValData.iteritems():
        for mediaType,mediaTypeData in artistIDData.media.media.items():
            modValMediaData.update({code: [artistID,media.album,media.url] for code,media in mediaTypeData.items()})
    mediaData.update(modValMediaData)
    if (modVal+1) % 10 == 0:
        print(f"ModVal = {modVal+1}")
        
df = DataFrame(mediaData).T
df.columns = ["ArtistID", "Name", "Ref"]
knownMedia.save(data=df)

## Download Data

In [ ]:
mio   = bandcamp.MusicDBIO(verbose=False,local=True,mkDirs=False)
webio = bandcamp.RawWebData(debug=False)

In [ ]:
useArtist = False
numMaster = 250

knownAlbumsData = knownMedia.get()
knownAlbumsData['IndexModVal'] = knownAlbumsData.index.map(mio.getModVal)
availableNames  = concat([artistIDDF.head(numMaster) for artistID,artistIDDF in knownAlbumsData.groupby(["ArtistID"])])
localAlbumsDict = localAlbums.get()
availableNames  = availableNames[~availableNames.index.isin(localAlbumsDict.keys())]
albumNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby(["IndexModVal"])})

print(f"# {db} Album Search Results")
print(f"#   Available Album IDs:  {knownAlbumsData.shape[0]}")
print(f"#   Known Album IDs:      {len(localAlbumsDict)}")
print(f"#   Albums To Download:   {availableNames.shape[0]}")

del availableNames
del localAlbumsDict
del knownAlbumsData

#   Albums To Download:   23457
#   Albums To Download:   20437
#   Albums To Download:   12137
#   Albums To Download:   4132

In [ ]:
def saveAlbumData(db, localAlbumsDict, searchedForErrors):
    print("="*150)
    print(f"Saving {len(localAlbumsDict)} {db} Albums Data")
    localAlbums.save(data=localAlbumsDict)
    print(f"Saving {len(searchedForErrors)} {db} Searched For Errors")
    errors.save(data=searchedForErrors)
    print("="*150)

In [ ]:
ts = Timestat("Getting {0} AlbumIDs".format(db))
tt = getTT(skipEOD=False)

n    = 0
maxN = 25000000
localAlbumsDict     = localAlbums.get()
searchedForErrors   = errors.get()
stop = False
nErrors = []
for groupModVal,modValData in albumNamesToGet.iteritems():
    if stop is True:
        break
    for j,(albumID,row) in enumerate(modValData.iterrows()):
        if len(nErrors) >= 5:
            for artistID in nErrors:
                print(f"del searchedForErrors['{artistID}']")
                stop=True
                break
        if any([dct.get(albumID) is not None for dct in [localAlbumsDict, searchedForErrors]]):
            continue
            
        artistID   = row["ArtistID"]
        albumName  = row["Name"]
        albumRef  = row["Ref"]

        print(f"{groupModVal: <8} |{j: <8} | {n: <8} | ", end="")
        try:
            response = webio.getAlbumData(albumName=albumName, albumRef=albumRef)
        except:
            print("Error ==> {0}".format(albumName))
            searchedForErrors[albumID] = True
            nErrors.append(albumID)
            webio.sleep(10)
            continue

        if not isinstance(response,bytes):
            print("Error ==> {0}".format(albumName))
            searchedForErrors[albumID] = True
            nErrors.append(albumID)
            webio.sleep(3.5)
            continue

        nErrors = []
        modVal=mio.mv.get(albumID)
        mio.data.saveRawArtistAlbumData(data=response, modval=modVal, dbID=albumID)
        localAlbumsDict[albumID] = True
        webio.sleep(4.5)
        n += 1
        nLastErrors = 0
        
        if n % 5 == 0:
            if tt.isFinished():
                stop=True
                break

        if n % 50 == 0:
            webio.sleep(5)
            
        if n % 100 == 0:
            saveAlbumData(db, localAlbumsDict, searchedForErrors)
            if tt.isFinished():
                stop=True
                break
            webio.wait(10.0)

        if n >= maxN:
            print("Breaking after {0} downloads...".format(maxN))
            stop=True
            break

ts.stop()
if True: saveAlbumData(db, localAlbumsDict, searchedForErrors)

In [ ]:
from os import getpid
getpid()

# Download Lists

## Download Starter

In [ ]:
from apiutils import WebIO
from ioutils import FileIO, HTMLIO
io  = FileIO()
hio = HTMLIO()
wio = WebIO()

In [ ]:
starter             = {}
starter["List"]     = "https://www.bandcamp.org/lists.php"
starter["Genre"]    = "https://www.bandcamp.org/genre.php"
#starterBestAlbum = "https://www.bandcamp.org/ratings/6-highest-rated/2023/1"
starter["Rating"]   = "https://www.bandcamp.org/ratings"
starter["Discover"] = "https://www.bandcamp.org/discover"
starter["Releases"] = "https://www.bandcamp.org/releases"
starter["MustHear"] = "https://www.bandcamp.org/must-hear"

savename = "../../sandbox/AOTYstarter.p"
starterData = io.get(savename)
for key,url in starter.items():
    if starterData.get(key) is not None:
        continue
    retval = wio.get(url)
    if retval.code == 200:
        print(key)
        starterData[key] = retval.data
    wio.sleep(3)
        
print(f"Saving data to {savename}")
io.save(idata=starterData, ifile=savename)

In [ ]:
useStarter = False
useSite    = True

if useStarter is True:
    aotyData = io.get("../../sandbox/AOTYstarter.p")
elif useSite is True:
    aotyData = io.get("../../sandbox/AOTYsiteData.p") | io.get("../../sandbox/AOTYsiteData2.p")
else:
    aotyData = {}
    
N  = len(aotyData)
ts = Timestat(f"Sorting {N} Site Refs")
refsData = {"List": {}, "Lists": {}, "Rating": {}, "Album": {}, "Artist": {}, "Discover": {}, "Genre": {}, "MustHear": {}, "Release": {}, "Spotify": {}, "Apple": {}, "Amazon": {}, "User": {}}
for n,(key,keyData) in enumerate(aotyData.items()):
    if (n+1) % 500 == 0 or (n+1) == 100:
        ts.update(n=n+1, N=N)
        
    bsdata = hio.get(keyData)
    refs = bsdata.findAll("a")
    for ref in refs:
        href = ref.get('href')
        if not isinstance(href,str):
            continue
        if href.startswith("/list/"):
            refsData["List"][href] = ref.text.strip()
        elif "lists.php" in href:
            refsData["Lists"][href] = ref.text.strip()
        elif href.startswith("/ratings/"):
            refsData["Rating"][href] = ref.text.strip()
        elif href.startswith("/artist/"):
            refsData["Artist"][href] = ref.text.strip()
        elif href.startswith("/album/"):
            refsData["Album"][href] = ref.text.strip()
        elif href.startswith("/discover/"):
            refsData["Discover"][href] = ref.text.strip()
        elif href.startswith("/genre/"):
            refsData["Genre"][href] = ref.text.strip()
        elif href.startswith("/must-hear/"):
            refsData["MustHear"][href] = ref.text.strip()
        elif "/releases/" in href:
            refsData["Release"][href] = ref.text.strip()
        elif "spotify.com" in href:
            refsData["Spotify"][href] = ref.text.strip()
        elif "apple.com" in href:
            refsData["Apple"][href] = ref.text.strip()
        elif "amazon.com" in href:
            refsData["Amazon"][href] = ref.text.strip()
        elif "/user/" in href:
            refsData["User"][href] = ref.text.strip()
        else:
            continue
            print(href,'\t|\t',ref.text)
            
ts.stop()

In [ ]:
savename = "../../sandbox/AOTYsiteRefs.p"
print(f"Saving data to {savename}")
for key,keyData in refsData.items():
    print(f"  {key: <20}{len(keyData)}")
io.save(idata=refsData, ifile=savename)

In [ ]:
siteData   = {}
artistData = {}
albumData  = {}
userData   = {}
otherData  = {}

if False:
    for ref,name in refsData["List"].items():
        if ref.startswith("/artist/"):
            artistData[ref] = name
        elif ref.startswith("/album/"):
            albumData[ref] = name
        elif ref.startswith("/user/"):
            userData[ref] = name
        elif ref.startswith("/"):
            siteData[ref] = name
        else:
            otherData[ref] = name
else:
    for key,keyData in refsData.items():
        for ref,name in keyData.items():
            if ref.startswith("/artist/"):
                artistData[ref] = name
            elif ref.startswith("/album/"):
                albumData[ref] = name
            elif ref.startswith("/user/"):
                userData[ref] = name
            elif ref.startswith("/"):
                if "/list/" in ref:
                    siteData[ref] = name
                else:
                    otherData[ref] = name
            else:
                otherData[ref] = name
            
print(f"Found {len(artistData)} Artist Refs")
print(f"Found {len(albumData)} Album Refs")
print(f"Found {len(userData)} User Refs")
print(f"Found {len(siteData)} Site Refs")
print(f"Found {len(otherData)} Other Refs")

In [ ]:
for year in range(1970,2005):
    key = f'/lists.php?y={year}'
    val = 'View More'
    siteData[key] = val

In [ ]:
savename1 = "../../sandbox/AOTYsiteData.p"
siteDataDownloads1 = io.get(savename1)
print(f"Found {len(siteDataDownloads1)} Previous Downloads")
savename2 = "../../sandbox/AOTYsiteData2.p"
siteDataDownloads2 = io.get(savename2)
print(f"Found {len(siteDataDownloads2)} Previous Downloads")
N = len(siteData)
ts = Timestat(f"Downloading {N} Site Refs")
for n,(ref,name) in enumerate(siteData.items()):
    url=f"https://www.bandcamp.org{ref}"
    if any([dct.get(ref) is not None for dct in [siteDataDownloads1,siteDataDownloads2]]):
        continue
    
    retval = wio.get(url)
    if retval.code == 200:
        print(f"{n: <6} | {N: <6} | {ref}")
        siteDataDownloads2[ref] = retval.data
    wio.sleep(3)
    
    if (n+1) % 25 == 0:
        ts.update(n=n+1,N=N)
        print(f"Saving {len(siteDataDownloads2)} data to {savename2}")
        io.save(idata=siteDataDownloads2, ifile=savename2)

ts.stop()
        
print(f"Saving data to {savename2}")
io.save(idata=siteDataDownloads2, ifile=savename2)

In [ ]:
siteDataDownloads = io.get("../../sandbox/AOTYsiteData.p")

In [ ]:
io.save(idata=siteDataDownloads, ifile=savename)

## Check For New Data

In [ ]:
from lib.bandcamp import MusicDBID
mid = MusicDBID()
mid.getAlbumID('/album/515536-beyonce-renaissance/critic-lists/?f=all&y=2022')

In [ ]:
df = DataFrame(Series(refsData["Album"])).reset_index().rename(columns={"index": "Ref", 0: "List"})
df["AlbumID"] = df["Ref"].map(mid.getAlbumID)
df = df[~df["AlbumID"].duplicated()]

In [ ]:
df = DataFrame(Series(refsData["Artist"])).reset_index().rename(columns={"index": "Ref", 0: "Name"})
df["ArtistID"] = df["Ref"].map(mid.getArtistID)
df = df[~df["ArtistID"].duplicated()]

In [ ]:
artistNames = searchArtists()

In [ ]:
df.index = df["ArtistID"]
df = df.drop(["ArtistID"], axis=1)
artistNames = concat([artistNames,df])
artistNames = artistNames[~artistNames.index.duplicated()]

In [ ]:
mio.data.saveSearchArtistData(data=artistNames)

# Backup

In [ ]:
from utils import StoreData, backup
from numpy import array_split
sd = StoreData("bandcamp", "Artist")
for modVals in array_split(range(100), 2):
    sd.mergeLocalData(modVals=modVals)
sd.mergeGlobalData()

In [ ]:
from time import sleep
sleep(200)
sd.mergeGlobalData()

In [ ]:
for modVal in range(67):
    srcDir = DirInfo(f"/Volumes/Piggy/Discog/artists-bandcamp/{modVal}/artists")
    files  = [FileInfo(ifile) for ifile in srcDir.getFiles()]
    files  = [finfo for finfo in files if finfo.basename.isdigit()]
    dstDir = DirInfo(f"/Users/tgadfort/Music/Discog/artists-bandcamp/{modVal}/artists")
    for srcFile in files:
        dstFile = dstDir.join(srcFile.name)
        srcFile.mvFile(dstFile)